In [0]:
!python --version

Python 3.7.3


In [0]:
# Start writing your code here...

In [0]:
from google.colab import files
uploaded = files.upload()

Saving DOITT_SUBWAY_STATION_01_13SEPT2010.csv to DOITT_SUBWAY_STATION_01_13SEPT2010.csv


In [0]:
import pandas as pd

In [0]:
stations=pd.read_csv('DOITT_SUBWAY_STATION_01_13SEPT2010.csv')

In [0]:
stations.head(1)

,URL,OBJECTID,NAME,long,lat
0,http://web.mta.info/nyct/service/,1,Astor Pl,-73.99107,40.730054


In [0]:
airbnb=pd.read_csv('airbnb_merge.csv')

In [0]:
pip install folium


In [0]:
!pip install geopandas

     |████████████████████████████████| 931kB 2.8MB/s 
     |████████████████████████████████| 10.4MB 6.3MB/s 
     |████████████████████████████████| 14.7MB 64.1MB/s 


In [0]:
import pandas as pd
import geopandas as gpd 
import matplotlib.pyplot as plt

In [0]:

!pip install shapely

In [0]:
import folium 
from shapely.ops import nearest_points
from shapely.geometry import LineString

In [0]:
def create_gdf(df, x="long", y="lat"):
    return gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[x], df[y]), crs={"init":"epsg:4326"})

In [0]:
def create_gd(df, x="longitude", y="latitude"):
    return gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[x], df[y]), crs={"init":"epsg:4326"})

In [0]:
stations_gdf = create_gdf(stations)

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [0]:
airbnb_gdf = create_gd(airbnb)

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [0]:
stations_gdf

,URL,OBJECTID,NAME,long,lat,geometry
0,http://web.mta.info/nyct/service/,1,Astor Pl,-73.991070,40.730054,POINT (-73.99107 40.73005)
1,http://web.mta.info/nyct/service/,2,Canal St,-74.000193,40.718803,POINT (-74.00019 40.71880)
2,http://web.mta.info/nyct/service/,3,50th St,-73.983849,40.761728,POINT (-73.98385 40.76173)
3,http://web.mta.info/nyct/service/,4,Bergen St,-73.974999,40.680862,POINT (-73.97500 40.68086)
4,http://web.mta.info/nyct/service/,5,Pennsylvania Ave,-73.894886,40.664714,POINT (-73.89489 40.66471)
...,...,...,...,...,...,...
468,http://web.mta.info/nyct/service/,469,Coney Island - Stillwell Av,-73.981236,40.577281,POINT (-73.98124 40.57728)
469,http://web.mta.info/nyct/service/,470,34th St - Hudson Yards,-74.002197,40.755446,POINT (-74.00220 40.75545)
470,http://web.mta.info/nyct/service/,641,72nd St,-73.958362,40.768803,POINT (-73.95836 40.76880)
471,http://web.mta.info/nyct/service/,642,86th St,-73.951771,40.777861,POINT (-73.95177 40.77786)


In [0]:
m = folium.Map([40.7128,-74.0060], tiles='CartoDb dark_matter')
locs_stations = zip(stations_gdf.lat, stations_gdf.long)
locs_points = zip(airbnb_gdf.latitude, airbnb_gdf.longitude)
for location in locs_stations:
    folium.CircleMarker(location=location, 
        color='red',   radius=4).add_to(m)
for location in locs_points:
    folium.CircleMarker(location=location, 
        color='white', radius=2).add_to(m)
m

In [0]:
def calculate_nearest(row, destination, val, col="geometry"):
    dest_unary = destination["geometry"].unary_union
    nearest_geom = nearest_points(row[col], dest_unary)
    match_geom = destination.loc[destination.geometry == nearest_geom[1]]
    match_value = match_geom[val].to_numpy()[0]
    return match_value

In [0]:
airbnb_gdf["nearest_geom"] = airbnb_gdf.apply(calculate_nearest, destination=stations_gdf, val="geometry", axis=1)

In [0]:
airbnb_gdf.head(2)

,Unnamed: 0,id_airbnb,host_id,borough,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,description,accommodates,zipcode,neighborhood,total_yelp_1,yelp_total_reviews_1,yelp_avg_rating_1,total_yelp_5,yelp_total_reviews_5,yelp_avg_rating_5,geometry,nearest_geom
0,0,2595,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,10,48,0.39,1,1,"Beautiful, spacious skylit studio in the heart...",1,10018,Chelsea and Clinton,5.0,974.0,4.0,99.0,48723.0,4.045455,POINT (-73.98377 40.75362),POINT (-73.98459100000001 40.754184)
1,1,3831,4869,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,295,4.67,1,1,Urban retreat: enjoy 500 s.f. floor in 1899 br...,3,11238,Central Brooklyn,NaN,NaN,NaN,39.0,4876.0,4.230769,POINT (-73.95976 40.68514),POINT (-73.95999 40.688889)


In [0]:
airbnb_gdf["nearest_station"] = airbnb_gdf.apply(calculate_nearest, destination=stations_gdf, val="NAME", axis=1)

In [0]:
airbnb_gdf.head(2)

,Unnamed: 0,id_airbnb,host_id,borough,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,description,accommodates,zipcode,neighborhood,total_yelp_1,yelp_total_reviews_1,yelp_avg_rating_1,total_yelp_5,yelp_total_reviews_5,yelp_avg_rating_5,geometry,nearest_geom,nearest_station,lat,lon
0,0,2595,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,10,48,0.39,1,1,"Beautiful, spacious skylit studio in the heart...",1,10018,Chelsea and Clinton,5.0,974.0,4.0,99.0,48723.0,4.045455,POINT (-73.98377 40.75362),POINT (-73.98459100000001 40.754184),42nd St - Bryant Pk,40.754184,-73.984591
1,1,3831,4869,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,295,4.67,1,1,Urban retreat: enjoy 500 s.f. floor in 1899 br...,3,11238,Central Brooklyn,NaN,NaN,NaN,39.0,4876.0,4.230769,POINT (-73.95976 40.68514),POINT (-73.95999 40.688889),Classon Ave,40.688889,-73.959990


In [0]:
airbnb_gdf['lat']=airbnb_gdf.nearest_geom.apply(lambda p: p.y)

In [0]:
airbnb_gdf['lon']=airbnb_gdf.nearest_geom.apply(lambda p: p.x)

In [0]:
from math import sin, cos, sqrt, atan2, radians

def compute_distance(lat1, lon1, lat2, lon2):

    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return distance

In [0]:
airbnb_gdf['distance_to_station'] = list(map(compute_distance, airbnb_gdf['latitude'], 
                            airbnb_gdf['longitude'], airbnb_gdf['lat'],
                           airbnb_gdf['lon']))

In [0]:
airbnb_gdf.head(3)

,Unnamed: 0,id_airbnb,host_id,borough,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,description,accommodates,zipcode,neighborhood,total_yelp_1,yelp_total_reviews_1,yelp_avg_rating_1,total_yelp_5,yelp_total_reviews_5,yelp_avg_rating_5,geometry,nearest_geom,nearest_station,lat,lon,distance_to_station
0,0,2595,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,10,48,0.39,1,1,"Beautiful, spacious skylit studio in the heart...",1,10018,Chelsea and Clinton,5.0,974.0,4.0,99.0,48723.0,4.045455,POINT (-73.98377 40.75362),POINT (-73.98459100000001 40.754184),42nd St - Bryant Pk,40.754184,-73.984591,0.093386
1,1,3831,4869,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,295,4.67,1,1,Urban retreat: enjoy 500 s.f. floor in 1899 br...,3,11238,Central Brooklyn,NaN,NaN,NaN,39.0,4876.0,4.230769,POINT (-73.95976 40.68514),POINT (-73.95999 40.688889),Classon Ave,40.688889,-73.959990,0.417452
2,2,5099,7322,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,78,0.60,1,19,My large 1 bedroom apartment has a true New Yo...,2,10016,Gramercy Park and Murray Hill,1.0,759.0,4.5,48.0,18016.0,4.156250,POINT (-73.97500 40.74767),POINT (-73.97604100000001 40.751431),Grand Central - 42nd St,40.751431,-73.976041,0.427433


In [0]:
airbnb_gdf.drop(['geometry','nearest_geom'], axis=1,inplace=True)

In [0]:
airbnb_gdf.to_csv('airbnb_w_station.csv')